In [6]:
import requests
import pandas as pd
import numpy as np
from hmmlearn.hmm import GaussianHMM
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Configuration
API_KEY = "ca72e792bc6e44b4be1230936250904"
CITY = "Lahore"
BASE_URL = f"http://api.weatherapi.com/v1/history.json?key={API_KEY}&q={CITY}&dt="

# Generate 100 days of past dates
dates = pd.date_range(end=pd.to_datetime('today'), periods=100).strftime('%Y-%m-%d')

# Initialize list to store weather data
weather_data = []

# Fetch historical weather data
print("Fetching weather data...")
for date in dates:
    url = BASE_URL + date
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        if 'forecast' in data:
            day_data = data['forecast']['forecastday'][0]['day']
            weather_data.append({
                "date": date,
                "temp_c": day_data.get("avgtemp_c"),
                "humidity": day_data.get("avghumidity"),
                "wind_kph": day_data.get("maxwind_kph"),
                "pressure_mb": day_data.get("avgvis_km"),
                "cloud": day_data.get("daily_chance_of_rain", 0),
                "visibility_km": day_data.get("avgvis_km")
            })

# Convert to DataFrame
weather_df = pd.DataFrame(weather_data)

# Drop rows with missing data
weather_df.dropna(inplace=True)

# Show data shape
print(f"\nData shape after cleanup: {weather_df.shape}")

# Feature matrix
X = weather_df[["temp_c", "humidity", "wind_kph", "pressure_mb", "cloud", "visibility_km"]].values

# Train Gaussian HMM
print("\nTraining Hidden Markov Model...")
model = GaussianHMM(n_components=4, covariance_type="full", n_iter=1000, random_state=42)
model.fit(X)

# Predict next days using the last known state
print("\nPredicting next day...")
last_state = model.predict([X[-1]])[0]
predicted_vals = []

for _ in range(7):
    next_state = np.random.choice(np.arange(model.n_components), p=model.transmat_[last_state])
    next_mean = model.means_[next_state]
    predicted_vals.append(next_mean)
    last_state = next_state

# Convert predictions to DataFrame
future_dates = pd.date_range(start=datetime.today() + timedelta(days=1), periods=7).strftime('%Y-%m-%d')
predicted_df = pd.DataFrame(predicted_vals, columns=["temp_c", "humidity", "wind_kph", "pressure_mb", "cloud", "visibility_km"])
predicted_df["date"] = future_dates

# Merge actual and predicted for visualization
weather_df["type"] = "Actual"
predicted_df["type"] = "Predicted"
full_df = pd.concat([weather_df[["date", "temp_c", "humidity", "wind_kph", "pressure_mb", "cloud", "visibility_km", "type"]],
                     predicted_df], ignore_index=True)

# Print forecast
print("Next-Day Weather Forecast (Predicted):\n")
for idx, row in predicted_df.iterrows():
        print(f"{row['date']}:")
        print(f"  Temperature     : {row['temp_c']:.2f} °C")
        print(f"  Humidity        : {row['humidity']:.2f} %")
        print(f"  Wind Speed      : {row['wind_kph']:.2f} kph")
        print(f"  Pressure        : {row['pressure_mb']:.2f} mb")
        print(f"  Cloudiness      : {row['cloud']:.2f} %")
        print(f"  Visibility      : {row['visibility_km']:.2f} km\n")


Fetching weather data...


Model is not converging.  Current: 747.1011103361336 is not greater than 747.1013665123764. Delta is -0.0002561762428285874



Data shape after cleanup: (99, 7)

Training Hidden Markov Model...

Predicting next day...
Next-Day Weather Forecast (Predicted):

2025-04-12:
  Temperature     : 19.07 °C
  Humidity        : 35.89 %
  Wind Speed      : 14.21 kph
  Pressure        : 9.96 mb
  Cloudiness      : 91.39 %
  Visibility      : 9.96 km

2025-04-13:
  Temperature     : 19.07 °C
  Humidity        : 35.89 %
  Wind Speed      : 14.21 kph
  Pressure        : 9.96 mb
  Cloudiness      : 91.39 %
  Visibility      : 9.96 km

2025-04-14:
  Temperature     : 15.83 °C
  Humidity        : 71.33 %
  Wind Speed      : 14.50 kph
  Pressure        : 8.43 mb
  Cloudiness      : 33.33 %
  Visibility      : 8.43 km

2025-04-15:
  Temperature     : 19.11 °C
  Humidity        : 24.84 %
  Wind Speed      : 12.81 kph
  Pressure        : 10.00 mb
  Cloudiness      : 0.00 %
  Visibility      : 10.00 km

2025-04-16:
  Temperature     : 19.11 °C
  Humidity        : 24.84 %
  Wind Speed      : 12.81 kph
  Pressure        : 10.00 mb
  C